In [83]:
import pandas as pd
import json
from numpy import NaN
from pyproj import Proj, transform
from bokeh.io import output_notebook
from bokeh.models import HoverTool, ColumnDataSource
from bokeh.plotting import figure, show
from bokeh.tile_providers import get_provider, Vendors
from bokeh.palettes import Category20
from bokeh.transform import factor_cmap

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def LongLat_to_EN(long, lat):
    try:
        easting, northing = transform(Proj(init='epsg:4326'), Proj(init='epsg:3857'), long, lat)
        return easting, northing
    except:
        return None, None

In [4]:
with open("quartiers.json", 'r', encoding="utf-8") as file:
    quartiers = json.load(file)

budget = pd.read_csv("budget.csv", sep = ';')

# Quartiers

In [5]:
q_id = []
q_nom = []
q_point = []
q_xshape = []
q_yshape = []

for field in quartiers:
    q = field['fields']
    q_id.append(q['nuquart'])
    q_nom.append(q['nom'])
    q_point.append(list(LongLat_to_EN(*q['geo_point_2d'][::-1])))
    
    coords = q['geo_shape']['coordinates']
    mesx = []
    mesy = []
    for level1 in coords :
        for c in level1:
            x,y = LongLat_to_EN(c[0],c[1])
            mesx.append(x)
            mesy.append(y)
    q_xshape.append(mesx)
    q_yshape.append(mesy)

In [43]:
df = pd.DataFrame({'q_id': q_id, 'q_nom': q_nom, 'q_point': q_point, 'q_x': q_xshape, 'q_y':q_yshape})
df = df.sort_values('q_id').reset_index(drop = True)
df.to_json("formated_quartiers.json")

# Budget

In [93]:
def bpToQuartier(chaine):
    try:
        list_q = chaine.split(' - ')
        num_q = int(list_q[0])
        quartier = ' - '.join(list_q[1:])
        serie = pd.Series((num_q, quartier))
    except:
        serie = pd.Series((NaN, NaN))
    return serie

In [94]:
def bpToYear(chaine):
    try:
        annee = chaine[24:28]
    except:
        annee = NaN
    return annee

In [107]:
bd = pd.DataFrame()

bd[['b_numero', 'b_libelle', 'b_description', 'b_realise', 'b_lien', 'b_photo']] = budget[['Numéro', 'Libellé', 'Description', 'Réalisé', 'Lien_page_projet', 'Photo']]
bd['b_year'] = budget['Année_Budget'].apply(bpToYear)
bd[['b_q_id', 'b_quartier']] = budget['Quartier'].apply(bpToQuartier)
bd[['b_x', 'b_y']] = budget.apply(lambda x: pd.Series(LongLat_to_EN(x['Longitude'], x['Latitude'])), axis = 1)

bd.to_csv("formated_budget.csv", index=False)

# Tests
## Test Quartiers

In [45]:
source = ColumnDataSource(df)

p = figure(x_range=(-195729.64378600637, -175690.50954609076), y_range=(6119069.396713561, 6131687.895850816), x_axis_type="mercator", y_axis_type="mercator")
p.add_tile(get_provider(Vendors.CARTODBPOSITRON))
hover_tool = HoverTool(tooltips=[( 'Nom',   '@q_nom')])

#Affichage des données
p.patches(xs="q_x",ys="q_y",source = source, color = factor_cmap('q_nom', palette=Category20[12], factors=q_nom),alpha=0.5)
p.add_tools(hover_tool)
show(p)

## Test Budget

In [119]:
b_source = ColumnDataSource(bd)
q = figure(x_range=(-195729.64378600637, -175690.50954609076), y_range=(6119069.396713561, 6131687.895850816), x_axis_type="mercator", y_axis_type="mercator")
q.add_tile(get_provider(Vendors.CARTODBPOSITRON))

b_hover = HoverTool(
    tooltips = [
        ('Titre', '@b_libelle'),
        ('Description', '@b_description'),
        ('Année', '@b_year')
    ]
)

q.circle('b_x', 'b_y', source = b_source, size = 8, color = factor_cmap('b_year', palette=Category20[4], factors=bd.b_year.dropna().unique().tolist()),alpha=0.5)
q.add_tools(b_hover)
show(q)

In [120]:
p.circle('b_x', 'b_y', source = b_source, size = 10, color = factor_cmap('b_year', palette=Category20[4], factors=bd.b_year.dropna().unique().tolist()),alpha=0.5)
p.add_tools(b_hover)

In [121]:
show(p)